In [0]:
import os, sys
sys.path.append("/home/cfriedline/ipynb/include_utils")
import include_utils as u
import gzip
import shutil
import tempfile
from ipyparallel import Client
import scandir
import glob

In [0]:
rc = Client(profile="sge")

In [0]:
dview = rc[:]
lview = rc.load_balanced_view()
len(dview)

In [0]:
rootdir = "/home/cfriedline/eckertlab/projects/burt/seq"

In [0]:
# organize files for demultiplexing with GBSX
for seqdir in ['160525', '160520']:
    os.chdir('{}/{}'.format(rootdir, seqdir))
    for i in range(8):
        d = "Burt{}".format(i)
        if not os.path.exists(d):
            os.mkdir(d)
        f = glob.glob("{}*.gz".format(d))
        if f:
            assert len(f) == 1
            shutil.move(f[0], d)

In [0]:
cd ~/eckertlab/projects/burt/seq

In [0]:
src_files = []
for seqdir in ['160525', '160520']:
    os.chdir('{}/{}'.format(rootdir, seqdir))
    files = !find . -name 'Burt*.gz'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        src_files.append(x)
src_files = sorted(src_files)

In [0]:
def build_cmd(fastq, bc):
    cmd = "/home/cfriedline/g/src/jdk1.8.0_92/bin/java -jar /home/cfriedline/g/src/GBSX/GBSX_v1.2.jar --Demultiplexer"
    return "{} -f1 {} -i {} -gzip true -rad true -mb 2".format(cmd, fastq, bc)

def write_qsub(workdir, cmd, label):
    with open(os.path.join(workdir, "run.sh"), "w") as o:
        o.write("""#!/bin/bash
#$ -N gbsx{0}
#$ -cwd
#$ -V
#$ -S /bin/bash
#$ -e gbsx{0}.err
#$ -o gbsx{0}.out
cd {1}
{2}
""".format(label, workdir, cmd))

for s in src_files:
    label = os.path.basename(s).split("Burt")[1].split("_")[0]
    bc_file = os.path.join(rootdir, "barcode_{}_gbsx.txt".format(label))
    workdir = os.path.dirname(s)
    cmd = build_cmd(s, bc_file)
    write_qsub(workdir, cmd, label)

In [0]:
cd $rootdir

### Submit jobs to SGE
```
cd /gpfs_fs/home/eckertlab/projects/burt/seq
find . -name "run.sh" | xargs chmod +x
find . -name "run.sh" -exec qsub {} \;
```